In [ ]:
require "nn";

In [ ]:
BATCH_SIZE = 1000
COLS = 56
ROWS = 96000
EPOCHS = 20

In [ ]:
--[[
coroutine for generating batches
--]]
coTrainBatchGen = coroutine.create(function()
    local currentEpoch = 1
    repeat
        local csvTarget = io.open('train_labels.csv', 'r')
        local header = csvTarget:read()
        for targetLine in csvTarget:lines('*l') do
            local l = targetLine:split(",")
            local filename = l[1]
            local target = l[2]
            print("target is " .. target)
            local csvFile = io.open('train/' .. filename)
            print("Reading " .. filename)
            for i = 1, 96000, 1000 do
                print("i is " .. i)
                local data = torch.Tensor(BATCH_SIZE, COLS)
                for j = 1, BATCH_SIZE do
                    local trainLine = csvFile:read()
                    local l2 = trainLine:split(",")
                    for key, val in ipairs(l2) do
                        data[j][key] = val
                    end
                end
                local batch = {data = data, label = target}
                coroutine.yield(batch)
            end
            csvFile:close()
        end
        csvTarget:close()
        currentEpoch = currentEpoch + 1
    until currentEpoch == EPOCHS
    end
)

In [ ]:
mcdcnn = nn.Sequential();
mc = nn.Parallel(2, 1);
for i = 1,COLS do
    local c = nn.Sequential()
    c:add(nn.TemporalConvolution(1, 16, 128)) -- in: 1000x1, out: 873x16
    c:add(nn.ReLU())
    c:add(nn.TemporalMaxPooling(2)) -- out: 873 / 2 = 436
    c:add(nn.TemporalConvolution(16, 32, 64)) -- in: 436x16, out: 373x32
    c:add(nn.ReLU())
    c:add(nn.TemporalMaxPooling(2)) -- 373 / 2 = 186
--     c:add(nn.TemporalConvolution(32, 64, 32)) -- in: 186x32, out: 155x64
--     c:add(nn.ReLU())
--     c:add(nn.TemporalMaxPooling(2)) -- 155 / 2 = 77
--     c:add(nn.TemporalConvolution(64, 128, 16)) -- in: 155x64, out: 140x128
--     c:add(nn.ReLU())
--     c:add(nn.TemporalMaxPooling(2)) -- 140 / 2 = 70
    c:add(nn.View(186*32))
    mc:add(c)
end
mcdcnn:add(mc)
mcdcnn:add(nn.Linear(186 * 32 * COLS, 732))
mcdcnn:add(nn.ReLU())
mcdcnn:add(nn.Linear(732, 2))
print(mcdcnn:__tostring());

In [ ]:
criterion = nn.BCECriterion()

In [ ]:
trainer = nn.StochasticGradient(mcdcnn, criterion)
trainer.learningRate = 0.001
trainer.maxIteration = EPOCHS

In [ ]:
repeat
    status, batch = coroutine.resume(coTrainBatchGen)
    setmetatable(batch, 
    {__index = function(t, i) 
                    return {t.data[i], t.label[i]} 
                end}
    );

    function batch:size() 
        return self.data:size(1) 
    end
--     mcdcnn:forward(batch.data)
    trainer:train(batch)
until status == false